In [40]:
#IMPORTAR BIBLIOTECAS UTILIZADAS

import numpy as np
import pandas as pd
import random
from collections import Counter
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

In [41]:
"""
    Classe para processar o dataset
    Obs: o dataset tem que ser para problemas de classificacao e nele tem que conter uma coluna Class
"""
import os
import sys
sys.path.insert(0, os.path.abspath('../Util'))
from dados import ProcessarDados

In [42]:
#IMPORTANDO DATASET

dataset = ProcessarDados('../dataset/norm_bin_10_FEATURES_M17_CM6b_TH199.csv')

In [43]:
# função que treina e testa o modelo armazenando as métricas
# retorna um dicionário cotendo os valores das métricas de cada rodada

def treinamento_teste(epocas = 10, k_folds = 5, exibir_matriz_confusao=False, exibir_metricas=False):
    
    #array para armazenar as das métricas de cada rodada
    resultados_accuracy = []
    resultados_precision = []
    resultados_recall = []
    resultados_f1 = []
    resultados_parametros = []
    
    #dicionário das métricas
    resultados_gerais = {}

    for i in range(epocas):
        #SEPARAR DADOS DE TREINO E TESTE MANTENDO A MESMA PROPORÇÃO  (80% E 20%)
        
        X_treino, X_teste, y_treino, y_teste = dataset.holdout(0.2)
        
        #ALTERANDO DIMENSÃO DE Y PARA D=1

        y_treino = y_treino.reshape(480,)
        y_teste = y_teste.reshape(120,)
        
        # realizando o grid search para encontrar o melhor alpha, 
        # considerando a acurácia (taxa de acerto)
        # aqui o método GridSearchCV é configurado para subdividir os dados de treino em k_folds
        rl = LogisticRegression(random_state = 0)
        grid_rl = GridSearchCV(rl, param_grid, cv=k_folds, scoring='accuracy', verbose=0)
        grid_rl.fit(X_treino, y_treino)

        # Treinando do modelo com os melhores parametros encontrados
        alpha_best = grid_rl.best_estimator_.tol

        rl = LogisticRegression(random_state = 0, tol = alpha_best)
        rl.fit(X_treino, y_treino)

        #testando o modelo
        y_pred = rl.predict(X_teste)
        
        if exibir_matriz_confusao:
            print(confusion_matrix(y_teste, y_pred))


        # calculado as metricas
        accuracy = metrics.accuracy_score(y_teste, y_pred)
        precision = metrics.precision_score(y_teste, y_pred)
        recall = metrics.recall_score(y_teste, y_pred)
        f1_score = 2 * (precision * recall) / (precision + recall)
        

        # armazenando as métricas
        resultados_accuracy.append(accuracy)
        resultados_precision.append(precision)
        resultados_recall.append(recall)
        resultados_f1.append(f1_score)

        best_parametros = "Alpha: "+ str(alpha_best);
        resultados_parametros.append(best_parametros)
        
        if exibir_metricas:
            print("Rodada: #",i)
            print(best_parametros)
            print("Accuracy:",accuracy)
            print("Precision:",precision)
            print("Recall:",recall)
            print("F1-Score:",f1_score)
            print("\n")

            
    resultados_gerais['accuracy'] = resultados_accuracy
    resultados_gerais['precision'] = resultados_precision
    resultados_gerais['recall'] = resultados_recall
    resultados_gerais['f1'] = resultados_f1
    
    return resultados_gerais

In [44]:
def tabelaMetricas(nome_modelo, dict_metricas, rodadas=False, salvarResultados=True):
        
    print ("============================================== "+nome_modelo+" =================================================")
    print ("=================================== TABELA DE MÉTRICAS DO MODELO ===================================")
    
    if(rodadas==False):
        print ("\t Accuracy \t|\t Precision \t|\t Recall \t|\t F1-Score")
        print ("      %.2f +- %.2f" % (np.mean(dict_metricas['accuracy'], axis=0), np.std(dict_metricas['accuracy'], axis=0)),end='       ')
        print ("      %.2f +- %.2f" % (np.mean(dict_metricas['precision'], axis=0), np.std(dict_metricas['precision'], axis=0)),end='    ')
        print ("      %.2f +- %.2f" % (np.mean(dict_metricas['recall'], axis=0), np.std(dict_metricas['recall'], axis=0)),end='       ')
        print ("      %.2f +- %.2f" % (np.mean(dict_metricas['f1'], axis=0), np.std(dict_metricas['f1'], axis=0)))
        print ("====================================================================================================")
        
    if(salvarResultados):
        # save to npy file
        np.save('../resultados/resultados_'+nome_modelo+'.npy', dict_metricas)

In [45]:
#DEFININDO OS PARÂMETROS


alpha_range = np.logspace(-3,0,25)
param_grid = {'tol': alpha_range}

epocas = 50
k_folds = 5
exibir_matriz_confusao = False
exibir_metricas = True

# TREINANDO O MODELO E OBTENDO AS MÉTRICAS

dict_metricas = treinamento_teste(epocas, k_folds, exibir_matriz_confusao, exibir_metricas)
tabelaMetricas('RL',dict_metricas)

Rodada: # 0
Alpha: 0.1
Accuracy: 0.8916666666666667
Precision: 0.9130434782608695
Recall: 0.9438202247191011
F1-Score: 0.9281767955801105


Rodada: # 1
Alpha: 0.001
Accuracy: 0.9333333333333333
Precision: 0.9263157894736842
Recall: 0.9887640449438202
F1-Score: 0.9565217391304348


Rodada: # 2
Alpha: 0.001
Accuracy: 0.8916666666666667
Precision: 0.9139784946236559
Recall: 0.9444444444444444
F1-Score: 0.9289617486338798


Rodada: # 3
Alpha: 0.001
Accuracy: 0.8916666666666667
Precision: 0.945054945054945
Recall: 0.9148936170212766
F1-Score: 0.9297297297297297


Rodada: # 4
Alpha: 0.001
Accuracy: 0.9083333333333333
Precision: 0.9072164948453608
Recall: 0.9777777777777777
F1-Score: 0.9411764705882353


Rodada: # 5
Alpha: 0.001
Accuracy: 0.8666666666666667
Precision: 0.9340659340659341
Recall: 0.8947368421052632
F1-Score: 0.913978494623656


Rodada: # 6
Alpha: 0.1778279410038923
Accuracy: 0.9083333333333333
Precision: 0.9191919191919192
Recall: 0.9680851063829787
F1-Score: 0.9430051813471503